### Install needed dependencies
- `xpress` to import the CPLEX solver

*Note: first we need to install `numpy` being a needed dependency of the previous libraries*


In [3]:
!pip install numpy

  Using cached numpy-2.1.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
    --------------------------------------- 0.3/12.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.9 MB 1.2 MB/s eta 0:00:11
   -- ------------------------------------- 0.8/12.9 MB 1.2 MB/s eta 0:00:11
   --- ------------------------------------ 1.0/12.9 MB 1.2 MB/s eta 0:00:10
   ---- ----------------------------------- 1.3/12.9 MB 1.2 MB/s eta 0:00:10
   ---- ----------------------------------- 1.6/12.9 MB 1.2 MB/s eta 0:00:10
   ---- ----------------------------------- 1.6/12.9 MB 1.2 MB/s eta 0:00:10
   ------ --------------------------------- 2.1/12.9 MB 1.2 MB/s eta 0:00:09
   ------- -------------------------------- 2.4/12.9 MB 1.2 MB/s eta 0:00:09
   -------- ------------------------------- 2.6/12.9 MB 1.2 MB/s eta 0:00:09
   -------- ------------------------------- 2.9/12.9 MB 1.2 MB/s eta 0:00:09
   --------- -------

In [7]:
!pip install xpress

  Using cached xpress-9.5.0-cp310-cp310-win_amd64.whl.metadata (5.5 kB)
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
  Using cached xpresslibs-9.5.0-cp38-abi3-win_amd64.whl.metadata (1.5 kB)
Using cached xpress-9.5.0-cp310-cp310-win_amd64.whl (2.3 MB)
Using cached xpresslibs-9.5.0-cp38-abi3-win_amd64.whl (45.9 MB)
Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3


### Example problem to test the xpress library was correctly installed

In [ ]:
import xpress as xp
p = xp.problem(name='example')  # problem name (optional)
x1 = p.addVariable(vartype=xp.integer, name='x1', lb=-10, ub=10)
x2 = p.addVariable(name='x2')
p.setObjective(x1**2 + 2*x2)      # objective function
p.addConstraint(x1 + 3*x2 >= 4)   # one or more constraints
p.optimize()
print ("solution: {0} = {1}; {2} = {3}".format (x1.name, p.getSolution(x1), x2.name, p.getSolution(x2)))

FICO Xpress v9.5.0, Community, solve started 20:51:02, Nov 27, 2024
Heap usage: 387KB (peak 387KB, 106KB system)
Minimizing MIQP myexample using up to 8 threads and up to 8087MB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
         1 rows            2 cols            2 elements         1 entities
         1 qobjelem
Presolved problem has:
         1 rows            2 cols            2 elements         1 entities
         1 qobjelem
Presolve finished in 0 seconds
Heap usage: 416KB (peak 431KB, 106KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  3.00e+00] / [ 1.00e+00,  3.00e+00]
  RHS and bounds [min,max] : [ 4.00e+00,  1.00e+01] / [ 4.00e+00,  1.00e+01]
  Objective      [min,max] : [ 2.00e+00,  2.00e+00] / [ 2.00e+00,  2.00e+00]
  Quadratic      [min,max] : [ 2.00e+00,  2.00e+00] / [ 2.00e+00,  2.00e+00]
Autoscaling ap